In [2]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 17: Chronospatial Computer ---</h2><p>The Historians push the button on their strange device, but this time, you all just feel like you're <a href="/2018/day/6">falling</a>.</p>
<p>"Situation critical", the device announces in a familiar voice. "Bootstrapping process failed. Initializing debugger...."</p>
<p>The small handheld device suddenly unfolds into an entire computer! The Historians look around nervously before one of them tosses it to you.</p>
<p>This seems to be a 3-bit computer: its program is a list of 3-bit numbers (0 through 7), like <code>0,1,2,3</code>. The computer also has three <em>registers</em> named <code>A</code>, <code>B</code>, and <code>C</code>, but these registers aren't limited to 3 bits and can instead hold any integer.</p>
<p>The computer knows <em>eight instructions</em>, each identified by a 3-bit number (called the instruction's <em>opcode</em>). Each instruction also reads the 3-bit number after it as an input; this is called its <em>operand</em>.</p>
<p>A number called the <em>instruction pointer</em> identifies the position in the program from which the next opcode will be read; it starts at <code>0</code>, pointing at the first 3-bit number in the program. Except for jump instructions, the instruction pointer increases by <code>2</code> after each instruction is processed (to move past the instruction's opcode and its operand). If the computer tries to read an opcode past the end of the program, it instead <em>halts</em>.</p>
<p>So, the program <code>0,1,2,3</code> would run the instruction whose opcode is <code>0</code> and pass it the operand <code>1</code>, then run the instruction having opcode <code>2</code> and pass it the operand <code>3</code>, then halt.</p>
<p>There are two types of operands; each instruction specifies the type of its operand. The value of a <em>literal operand</em> is the operand itself. For example, the value of the literal operand <code>7</code> is the number <code>7</code>. The value of a <em>combo operand</em> can be found as follows:</p>
<ul>
<li>Combo operands <code>0</code> through <code>3</code> represent literal values <code>0</code> through <code>3</code>.</li>
<li>Combo operand <code>4</code> represents the value of register <code>A</code>.</li>
<li>Combo operand <code>5</code> represents the value of register <code>B</code>.</li>
<li>Combo operand <code>6</code> represents the value of register <code>C</code>.</li>
<li>Combo operand <code>7</code> is reserved and will not appear in valid programs.</li>
</ul>
<p>The eight instructions are as follows:</p>
<p>The <code><em>adv</em></code> instruction (opcode <code><em>0</em></code>) performs <em>division</em>. The numerator is the value in the <code>A</code> register. The denominator is found by raising 2 to the power of the instruction's <em>combo</em> operand. (So, an operand of <code>2</code> would divide <code>A</code> by <code>4</code> (<code>2^2</code>); an operand of <code>5</code> would divide <code>A</code> by <code>2^B</code>.) The result of the division operation is <em>truncated</em> to an integer and then written to the <code>A</code> register.</p>
<p>The <code><em>bxl</em></code> instruction (opcode <code><em>1</em></code>) calculates the <a href="https://en.wikipedia.org/wiki/Bitwise_operation#XOR" target="_blank">bitwise XOR</a> of register <code>B</code> and the instruction's <em>literal</em> operand, then stores the result in register <code>B</code>.</p>
<p>The <code><em>bst</em></code> instruction (opcode <code><em>2</em></code>) calculates the value of its <em>combo</em> operand <a href="https://en.wikipedia.org/wiki/Modulo" target="_blank">modulo</a> 8 (thereby keeping only its lowest 3 bits), then writes that value to the <code>B</code> register.</p>
<p>The <code><em>jnz</em></code> instruction (opcode <code><em>3</em></code>) does <em>nothing</em> if the <code>A</code> register is <code>0</code>. However, if the <code>A</code> register is <em>not zero</em>, it <span title="The instruction does this using a little trampoline."><em>jumps</em></span> by setting the instruction pointer to the value of its <em>literal</em> operand; if this instruction jumps, the instruction pointer is <em>not</em> increased by <code>2</code> after this instruction.</p>
<p>The <code><em>bxc</em></code> instruction (opcode <code><em>4</em></code>) calculates the <em>bitwise XOR</em> of register <code>B</code> and register <code>C</code>, then stores the result in register <code>B</code>. (For legacy reasons, this instruction reads an operand but <em>ignores</em> it.)</p>
<p>The <code><em>out</em></code> instruction (opcode <code><em>5</em></code>) calculates the value of its <em>combo</em> operand modulo 8, then <em>outputs</em> that value. (If a program outputs multiple values, they are separated by commas.)</p>
<p>The <code><em>bdv</em></code> instruction (opcode <code><em>6</em></code>) works exactly like the <code>adv</code> instruction except that the result is stored in the <em><code>B</code> register</em>. (The numerator is still read from the <code>A</code> register.)</p>
<p>The <code><em>cdv</em></code> instruction (opcode <code><em>7</em></code>) works exactly like the <code>adv</code> instruction except that the result is stored in the <em><code>C</code> register</em>. (The numerator is still read from the <code>A</code> register.)</p>
<p>Here are some examples of instruction operation:</p>
<ul>
<li>If register <code>C</code> contains <code>9</code>, the program <code>2,6</code> would set register <code>B</code> to <code>1</code>.</li>
<li>If register <code>A</code> contains <code>10</code>, the program <code>5,0,5,1,5,4</code> would output <code>0,1,2</code>.</li>
<li>If register <code>A</code> contains <code>2024</code>, the program <code>0,1,5,4,3,0</code> would output <code>4,2,5,6,7,7,7,7,3,1,0</code> and leave <code>0</code> in register <code>A</code>.</li>
<li>If register <code>B</code> contains <code>29</code>, the program <code>1,7</code> would set register <code>B</code> to <code>26</code>.</li>
<li>If register <code>B</code> contains <code>2024</code> and register <code>C</code> contains <code>43690</code>, the program <code>4,0</code> would set register <code>B</code> to <code>44354</code>.</li>
</ul>
<p>The Historians' strange device has finished initializing its debugger and is displaying some <em>information about the program it is trying to run</em> (your puzzle input). For example:</p>
<pre><code>Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0
</code></pre>

<p>Your first task is to <em>determine what the program is trying to output</em>. To do this, initialize the registers to the given values, then run the given program, collecting any output produced by <code>out</code> instructions. (Always join the values produced by <code>out</code> instructions with commas.) After the above program halts, its final output will be <code><em>4,6,3,5,6,3,5,2,1,0</em></code>.</p>
<p>Using the information provided by the debugger, initialize the registers to the given values, then run the program. Once it halts, <em>what do you get if you use commas to join the values it output into a single string?</em></p>
</article>


In [3]:
from typing import Self


tests = [
    {
        "name": "If register C contains 9, the program 2,6 would set register B to 1.",
        "s": """
            Register A: 0
            Register B: 0
            Register C: 9

            Program: 2,6
        """,
        "expected_reg": "B 1",
        "expected": "",
    },
    {
        "name": "If register A contains 10, the program 5,0,5,1,5,4 would output 0,1,2.",
        "s": """
            Register A: 10
            Register B: 0
            Register C: 0

            Program: 5,0,5,1,5,4
        """,
        "expected": "0,1,2",
    },
    {
        "name": "If register A contains 2024, the program 0,1,5,4,3,0 would output 4,2,5,6,7,7,7,7,3,1,0 and leave 0 in register A",
        "s": """
            Register A: 2024
            Register B: 0
            Register C: 0

            Program: 0,1,5,4,3,0
        """,
        "expected_reg": "A 0",
        "expected": "4,2,5,6,7,7,7,7,3,1,0",
    },
    {
        "name": "If register B contains 29, the program 1,7 would set register B to 26.",
        "s": """
            Register A: 0
            Register B: 29
            Register C: 0

            Program: 1,7
        """,
        "expected_reg": "B 26",
        "expected": "",
    },
    {
        "name": "If register B contains 2024 and register C contains 43690, the program 4,0 would set register B to 44354.",
        "s": """
            Register A: 0
            Register B: 2024
            Register C: 43690

            Program: 4,0
        """,
        "expected_reg": "B 44354",
        "expected": "",
    },
    {
        "name": "Example",
        "s": """
            Register A: 729
            Register B: 0
            Register C: 0

            Program: 0,1,5,4,3,0
        """,
        "expected": "4,6,3,5,6,3,5,2,1,0",
    },
]


class Computer(Str):
    def __init__(self) -> None:
        self.registers = {"A": 0, "B": 0, "C": 0}
        self.ip = 0
        self.program = None
        self.output = []

    def run_program(self, s: str, reset: bool = True) -> None:
        if reset:
            self._initialize(s)

        while self.ip < len(self.program):
            self.process_opcode(self.program[self.ip])

    def _initialize(self, s: str) -> None:
        a, b, c, self.program = self._parse(s)
        self.registers = {"A": a, "B": b, "C": c}
        self.output.clear()
        self.ip = 0

    def combo_arguement(self, combo_argument: int) -> int:
        if 0 <= combo_argument <= 3:
            # Combo operands 0 through 3 represent literal values 0 through 3.
            return combo_argument

        if 4 <= combo_argument <= 6:
            # Combo operand 4 represents the value of register A.
            # Combo operand 5 represents the value of register B.
            # Combo operand 6 represents the value of register C.
            return self.registers["ABC"[combo_argument - 4]]

        # Combo operand 7 is reserved and will not appear in valid programs.
        raise ValueError(
            "Combo operand 7 is reserved and will not appear in valid programs."
        )

    def process_opcode(self, opcode: int) -> None:
        match opcode:
            case 0:
                # The adv instruction (opcode 0) performs division.
                # The numerator is the value in the A register.
                # The denominator is found by raising 2 to the power of the instruction's combo operand.
                # The result of the division operation is truncated to an integer and then written to the A register.
                self.registers["A"] //= 2 ** self.combo_arguement(
                    self.program[self.ip + 1]
                )
                self.ip += 2
            case 1:
                # The bxl instruction (opcode 1) calculates the bitwise XOR of register B
                # then stores the result in register B.
                self.registers["B"] ^= self.program[self.ip + 1]
                self.ip += 2
            case 2:
                # The bst instruction (opcode 2) calculates the value of its combo operand modulo 8
                # then writes that value to the B register.
                self.registers["B"] = (
                    self.combo_arguement(self.program[self.ip + 1]) % 8
                )
                self.ip += 2
            case 3:
                # The jnz instruction (opcode 3) does nothing if the A register is 0.
                # However, if the A register is not zero,
                # it jumps by setting the instruction pointer to the value of
                # its literal operand;
                # if this instruction jumps, the instruction pointer is not
                # increased by 2 after this instruction.
                if self.registers["A"]:
                    self.ip = self.program[self.ip + 1]
                else:
                    self.ip += 2
            case 4:
                # The bxc instruction (opcode 4) calculates the bitwise XOR
                # of register B and register C,
                # then stores the result in register B.
                # (For legacy reasons, this instruction reads an operand but ignores it.)
                self.registers["B"] ^= self.registers["C"]
                self.ip += 2
            case 5:
                # The out instruction (opcode 5) calculates the value of its combo operand modulo 8,
                # then outputs that value. (If a program outputs multiple values, they are separated by commas.)
                self.output.append(self.combo_arguement(self.program[self.ip + 1]) % 8)
                self.ip += 2
            case 6:
                # The bdv instruction (opcode 6) works exactly like the adv instruction except that the result is
                # stored in the B register. (The numerator is still read from the A register.)
                self.registers["B"] = self.registers["A"] // 2 ** self.combo_arguement(
                    self.program[self.ip + 1]
                )
                self.ip += 2
            case 7:
                # The cdv instruction (opcode 7) works exactly like the adv instruction
                # except that the result is stored in the C register.
                #  (The numerator is still read from the A register.)
                self.registers["C"] = self.registers["A"] // 2 ** self.combo_arguement(
                    self.program[self.ip + 1]
                )
                self.ip += 2

    def str(self) -> str:
        return ",".join(map(str, self.output))

    @classmethod
    def _parse(cls, s: str) -> tuple[int, int, int, list[int]]:
        a, b, c, *program = map(int, re.findall(r"\d+", s))
        return a, b, c, program

    @classmethod
    def partI(cls, s: str) -> tuple[Self, str]:
        c = cls()
        c.run_program(s)
        out = c.str()
        return c, out


@test(tests=tests[:])
def partI_test(s: str, expected_reg: str | None = None) -> int:
    c, out = Computer.partI(s)
    if expected_reg is not None:
        register, val = expected_reg.split()
        assert c.registers[register] == int(val)
    return out


Test If register C contains 9, the program 2,6 would set register B to 1. passed, for partI_test.
Test If register A contains 10, the program 5,0,5,1,5,4 would output 0,1,2. passed, for partI_test.
Test If register A contains 2024, the program 0,1,5,4,3,0 would output 4,2,5,6,7,7,7,7,3,1,0 and leave 0 in register A passed, for partI_test.
Test If register B contains 29, the program 1,7 would set register B to 26. passed, for partI_test.
Test If register B contains 2024 and register C contains 43690, the program 4,0 would set register B to 44354. passed, for partI_test.
Test Example passed, for partI_test.
Success


In [4]:
from more_itertools import last


with open("../input/day17.txt") as f:
    puzzle = f.read()

print(f"Part I: {last(Computer.partI(puzzle))}")

Part I: 7,3,0,5,7,1,4,0,5


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>130536</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Digging deeper in the device's manual, you discover the problem: this program is supposed to <em>output another copy of the program</em>! Unfortunately, the value in register <code>A</code> seems to have been corrupted. You'll need to find a new value to which you can initialize register <code>A</code> so that the program's output instructions produce an exact copy of the program itself.</p>
<p>For example:</p>
<pre><code>Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0
</code></pre>

<p>This program outputs a copy of itself if register <code>A</code> is instead initialized to <code><em>117440</em></code>. (The original initial value of register <code>A</code>, <code>2024</code>, is ignored.)</p>
<p><em>What is the lowest positive initial value for register <code>A</code> that causes the program to output a copy of itself?</em></p>
</article>


In [ ]:
tests = [
    {
        "name": "Example",
        "program": [0, 3, 5, 4, 3, 0],
        "expected": 117440,
    },
]


def run_program(a: int, b: int, c: int, program: list[int]) -> list[int]:
    """Executes the 3-bit computer program for a given register A."""
    ip = 0
    output = []
    while ip < len(program):
        opcode = program[ip]
        operand = program[ip + 1]

        # Get combo operand value
        combo = operand
        if operand == 4:
            combo = a
        elif operand == 5:
            combo = b
        elif operand == 6:
            combo = c

        if opcode == 0:  # adv
            a >>= combo
        elif opcode == 1:  # bxl
            b ^= operand
        elif opcode == 2:  # bst
            b = combo % 8
        elif opcode == 3:  # jnz
            if a != 0:
                ip = operand
                continue
        elif opcode == 4:  # bxc
            b ^= c
        elif opcode == 5:  # out
            output.append(combo % 8)
        elif opcode == 6:  # bdv
            b = a >> combo
        elif opcode == 7:  # cdv
            c = a >> combo
        ip += 2
    return output


def find_a(program, target_index, current_a):
    """Recursive backtracking to find the smallest A."""
    if target_index < 0:
        return current_a

    for i in range(8):
        # Try appending 3 new bits to the current A
        next_a = (current_a << 3) | i
        # Simulate and check if this matches the tail of the program
        output = run_program(next_a, 0, 0, program)
        if output == program[target_index:]:
            # Recurse for the next instruction
            result = find_a(program, target_index - 1, next_a)
            if result is not None:
                return result
    return None


# Load your specific program here (example values below)
# program = [2, 4, 1, 1, 7, 5, 1, 5, 4, 0, 5, 5, 0, 3, 3, 0]
# result = find_a(program, len(program) - 1, 0)
# print(f"Smallest Register A: {result}")
@test(tests=tests)
def test_partII(program: list[int]) -> int:
    return find_a(program, len(program) - 1, 0)


Test Example passed, for test_partII.
Success


In [11]:
program = [2, 4, 1, 1, 7, 5, 4, 6, 0, 3, 1, 4, 5, 5, 3, 0]
result = find_a(program, len(program) - 1, 0)
print(f"PartII: {result}")

PartII: 202972175280682


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>202972175280682</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
